In [7]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import pandas as pd

In [9]:
# upload files
csv_file_path = "Aftertest _Questionnaire.csv"
data_afterquestionnaire = pd.read_csv(csv_file_path, usecols=[i for i in range(1,24)], header = 0, names = ['Subject Id', 'Group Id', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10','GQ1', 'GQ2', 'GQ3', 'GQ4', 'GQ5', 'GQ6', 'GQ7', 'FQ1', 'FQ2', 'FQ3', 'Comments'])
data_afterquestionnaire.rename(columns={"Subject Id\n":"Subject Id"}, inplace=True)

data_afterquestionnaire.head()
csv_file_path1 = "Beforetest_Questionnaire.csv"
data_beforequestionnaire = pd.read_csv(csv_file_path1,usecols=[i for i in range(2,15)],header = 0, names=['Subject Id', 'Group Id', 'Age', 'Gender', 'Nationality', 'OMSI', 'Task1', 'Task2', 'Task3', 'Task4', 'Task5', 'Task6', 'Task7'])
data_beforequestionnaire.loc[data_beforequestionnaire['Task5'] == 0, 'Task5'] = 6


csv_file_path2 = "AlternativeTestQuestionnaire.csv"
data_alternativequestionnaire = pd.read_csv(csv_file_path2,usecols=[i for i in range(1,17)],header = 0, names=['Subject Id', 'Group Id', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'FQ1', 'FQ2', 'FQ3', 'Comments'])
data_alternativequestionnaire.head()

csv_file_path2 = "followup-test.csv"
data_followup = pd.read_csv(csv_file_path2,usecols=[i for i in range(0,8)],header = 0, names=['Subject Id','Task1', 'Task2', 'Task3', 'Task4', 'Task5', 'Task6', 'Task7'])
data_followup.head()

merged = data_beforequestionnaire.merge(data_afterquestionnaire, on=['Subject Id', 'Group Id'])

print(merged)

    Subject Id  Group Id  Age  Gender      Nationality  OMSI  Task1  Task2  \
0            1         1   27  Female            China   157      1      1   
1            2         1   33    Male          British   893      1      1   
2            3         1   25  Female           Polish   171      1      1   
3            4         2   30    Male       Venezuelan   816      1      2   
4            5         2   23    Male           Danish   132      1      2   
5            6         2   25  Female  Danish/Egyptian   192      1      2   
6            9         3   33  Female            Greek    96      1      1   
7            8         3   33    Male   Northern irish    99      1      1   
8            7         3   34    Male           Danish   454      1      1   
9           10         4   24    Male           Danish    99      1      1   
10          12         4   32    Male           Danish   367      1      1   
11          11         4   25    Male         American   583    

In [10]:
import pandas as pd
from scipy.stats import pearsonr

# Define question keys
odd_questions = ['Q1', 'Q3', 'Q5', 'Q7', 'Q9']
even_questions = ['Q2', 'Q4', 'Q6', 'Q8', 'Q10']

# --- Process SUS_data ---
SUS_data = data_afterquestionnaire[['Subject Id', 'Group Id'] + odd_questions + even_questions].copy()

for q in odd_questions:
    SUS_data[q] = SUS_data[q] - 1
for q in even_questions:
    SUS_data[q] = 5 - SUS_data[q]

SUS_data['SUS'] = SUS_data[odd_questions + even_questions].sum(axis=1) * 2.5

# --- Process SUS_data_alt ---
SUS_data_alt = data_alternativequestionnaire[['Subject Id', 'Group Id'] + odd_questions + even_questions].copy()

for q in odd_questions:
    SUS_data_alt[q] = SUS_data_alt[q] - 1
for q in even_questions:
    SUS_data_alt[q] = 5 - SUS_data_alt[q]

SUS_data_alt['SUS'] = SUS_data_alt[odd_questions + even_questions].sum(axis=1) * 2.5

# --- Combine both SUS datasets ---
combined_sus = pd.concat([SUS_data[['Subject Id', 'SUS']], SUS_data_alt[['Subject Id', 'SUS']]], ignore_index=True)

# --- Get OMSI scores from before-questionnaire data ---
omsi_data = data_beforequestionnaire[['Subject Id', 'OMSI']].copy()
omsi_data['OMSI'] = pd.to_numeric(omsi_data['OMSI'], errors='coerce')

# --- Merge on Subject Id ---
merged_df = pd.merge(combined_sus, omsi_data, on='Subject Id')

# Drop any missing values
merged_df = merged_df.dropna()

print(merged_df)

# --- Run covariance test ---
covariance = merged_df.cov().loc['OMSI', 'SUS']
print(f"Covariance between OMSI and SUS: {covariance}")

# --- Pearson correlation test ---
corr, p_value = pearsonr(merged_df['OMSI'], merged_df['SUS'])
print(f"Pearson correlation: {corr:.3f}")
print(f"P-value: {p_value:.4f}")

# pearson cor = 0.076
# p-value = 0.7296


    Subject Id    SUS  OMSI
0            3   65.0   171
1            1   45.0   157
2            2   62.5   893
3            5   77.5   132
4            6   80.0   192
5            4   92.5   816
6            8   65.0    99
7            9   97.5    96
8            7   65.0   454
9           10   55.0    99
10          12   32.5   367
11          11   47.5   583
12          12   40.0   367
13           5   92.5   132
14           2   85.0   893
15           4   90.0   816
16          11   80.0   583
17           8  100.0    99
18           3   77.5   171
19           7   80.0   454
20          10   37.5    99
21           6   90.0   192
22           9   80.0    96
Covariance between OMSI and SUS: 439.2687747035572
Pearson correlation: 0.076
P-value: 0.7296


In [11]:
import pandas as pd
from scipy.stats import pearsonr

# Define question keys
odd_questions = ['Q1', 'Q3', 'Q5', 'Q7', 'Q9']
even_questions = ['Q2', 'Q4', 'Q6', 'Q8', 'Q10']

# --- Process SUS_data ---
SUS_data = data_afterquestionnaire[['Subject Id', 'Group Id'] + odd_questions + even_questions].copy()

for q in odd_questions:
    SUS_data[q] = SUS_data[q] - 1
for q in even_questions:
    SUS_data[q] = 5 - SUS_data[q]

SUS_data['SUS'] = SUS_data[odd_questions + even_questions].sum(axis=1) * 2.5

# --- Process SUS_data_alt ---
SUS_data_alt = data_alternativequestionnaire[['Subject Id', 'Group Id'] + odd_questions + even_questions].copy()

for q in odd_questions:
    SUS_data_alt[q] = SUS_data_alt[q] - 1
for q in even_questions:
    SUS_data_alt[q] = 5 - SUS_data_alt[q]

SUS_data_alt['SUS'] = SUS_data_alt[odd_questions + even_questions].sum(axis=1) * 2.5

# --- Combine both SUS datasets ---
combined_sus = pd.concat([SUS_data[['Subject Id', 'SUS']], SUS_data_alt[['Subject Id', 'SUS']]], ignore_index=True)

# --- Get OMSI scores from before-questionnaire data ---
omsi_data = data_beforequestionnaire[['Subject Id', 'OMSI']].copy()
omsi_data['OMSI'] = pd.to_numeric(omsi_data['OMSI'], errors='coerce')

# --- Merge on Subject Id ---
merged_df = pd.merge(combined_sus, omsi_data, on='Subject Id')

# Drop any missing values
merged_df = merged_df.dropna()

print(merged_df)

# --- Run covariance test ---
covariance = merged_df.cov().loc['OMSI', 'SUS']
print(f"Covariance between OMSI and SUS: {covariance}")

# --- Pearson correlation test ---
corr, p_value = pearsonr(merged_df['OMSI'], merged_df['SUS'])
print(f"Pearson correlation: {corr:.3f}")
print(f"P-value: {p_value:.4f}")

# pearson cor = 0.076
# p-value = 0.7296


    Subject Id    SUS  OMSI
0            3   65.0   171
1            1   45.0   157
2            2   62.5   893
3            5   77.5   132
4            6   80.0   192
5            4   92.5   816
6            8   65.0    99
7            9   97.5    96
8            7   65.0   454
9           10   55.0    99
10          12   32.5   367
11          11   47.5   583
12          12   40.0   367
13           5   92.5   132
14           2   85.0   893
15           4   90.0   816
16          11   80.0   583
17           8  100.0    99
18           3   77.5   171
19           7   80.0   454
20          10   37.5    99
21           6   90.0   192
22           9   80.0    96
Covariance between OMSI and SUS: 439.2687747035572
Pearson correlation: 0.076
P-value: 0.7296
